# GPT-2 SFT + DPO Unified Pipeline

This notebook fine-tunes a GPT-2 model with Supervised Fine-Tuning (SFT) and then aligns it with Direct Preference Optimization (DPO).

Template: We mimic the GPT-2 instruction formatting used in your SFT notebook:

- Instruction/Response format (no special chat tokens)
- Text = "### Instruction:\n{question}\n\n### Response:\n{answer}{eos}"

## 1. Setup

In [ ]:
%%capture
!pip -q install datasets transformers trl peft bitsandbytes accelerate colorama

In [ ]:
import os, warnings, random
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DPOTrainer, DPOConfig
from colorama import Fore, Style

warnings.filterwarnings('ignore')
device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
print(Fore.CYAN + f'Using device: {device}' + Style.RESET_ALL)

## 2. Paths & Base Model

In [ ]:
# Adjust these paths as needed
BASE_MODEL = 'Savoxism/gpt2-large-continued-pretraining'  # Your continued-pretrained GPT-2
SFT_INPUT_JSON = 'instructions.json'  # list of {question, answer}
DPO_JSONL = 'DPO/data/dpo_train.jsonl'  # jsonl lines: {prompt, chosen, rejected}

SFT_OUTPUT_DIR = 'outputs/gpt2_sft'
DPO_OUTPUT_DIR = 'outputs/gpt2_dpo'
os.makedirs('outputs', exist_ok=True)

## 3. Load SFT Dataset

In [ ]:
# Expect a local JSON array with: [{'question': str, 'answer': str}, ...]
sft_ds = load_dataset('json', data_files=SFT_INPUT_JSON, split='train')
print(Fore.YELLOW + f'SFT samples: {len(sft_ds)}' + Style.RESET_ALL)
print(sft_ds[0])

## 4. Tokenizer & Model (GPT-2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

use_bnb = torch.cuda.is_available()  # enable 4-bit only on CUDA
quant_config = None

if use_bnb:
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config if use_bnb else None,
    device_map='auto' if use_bnb else None,
)
model.config.pad_token_id = model.config.eos_token_id
if use_bnb:
    model = prepare_model_for_kbit_training(model)
print(model.__class__.__name__, 'loaded')

## 5. GPT-2 Instruction Template Formatter (mimic SFT notebook)

In [ ]:
def format_gpt2_instruction_template(batch, tokenizer):
    samples = []
    questions = batch['question']
    answers = batch['answer']
    for q, a in zip(questions, answers):
        txt = f'### Instruction:\n{q}\n\n### Response:\n{a}{tokenizer.eos_token}'
        samples.append(txt)
    return {
        'instruction': questions,
        'response': answers,
        'text': samples,
    }

sft_train = sft_ds.map(lambda x: format_gpt2_instruction_template(x, tokenizer), batched=True)
sft_train = sft_train.remove_columns([c for c in sft_train.column_names if c not in ['text']])
print(Fore.LIGHTMAGENTA_EX + sft_train[0]['text'][:160] + '...' + Style.RESET_ALL)

## 6. SFT with LoRA (GPT-2 attention targets)

In [ ]:
peft_config = LoraConfig(
    r=2,
    lora_alpha=4,
    lora_dropout=0.05,
    target_modules=['c_attn', 'c_proj'],
    task_type='CAUSAL_LM',
 )

sft_args = SFTConfig(
    output_dir=SFT_OUTPUT_DIR,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=50, # adjust this
    save_steps=1000, # adjust this
    max_seq_length=256,
    remove_unused_columns=False,
)

sft_trainer = SFTTrainer(
    model=model,
    args=sft_args,
    peft_config=peft_config,
    tokenizer=tokenizer,
    train_dataset=sft_train,
    dataset_text_field='text',
 )
sft_trainer.train()
sft_trainer.save_model(SFT_OUTPUT_DIR)

## 7. Load DPO Preference Dataset

In [ ]:
# Expect jsonl with lines: { 'prompt': str, 'chosen': str, 'rejected': str }
dpo_ds = load_dataset('json', data_files=DPO_JSONL, split='train')
print(Fore.YELLOW + f'DPO samples: {len(dpo_ds)}' + Style.RESET_ALL)
print(dpo_ds[0])

## 8. Convert Preference Prompts to GPT-2 Template

In [ ]:
def to_gpt2_prompt(prompt: str) -> str:
    return f'### Instruction:\n{prompt}\n\n### Response:\n'

def map_dpo_record(ex):
    p = to_gpt2_prompt(ex['prompt'])
    return {
        'prompt': p,
        'chosen': ex['chosen'],
        'rejected': ex['rejected'],
    }

dpo_ready = dpo_ds.map(map_dpo_record)
print(dpo_ready[0]['prompt'][:120])
print('chosen:', dpo_ready[0]['chosen'][:100])
print('rejected:', dpo_ready[0]['rejected'][:100])

## 9. DPO Training (reuse LoRA alignment)

In [ ]:
# Continue from SFT-adapted model in memory
dpo_args = DPOConfig(
    output_dir=DPO_OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    logging_steps=50,
    max_length=256,
)

dpo_trainer = DPOTrainer(
    model=sft_trainer.model,
    args=dpo_args,
    train_dataset=dpo_ready,
    processing_class=tokenizer,
    peft_config=peft_config,
)

dpo_trainer.train()
dpo_trainer.save_model(DPO_OUTPUT_DIR)

## 10. Inference

In [ ]:
def gpt2_infer(model, tokenizer, instruction: str, max_new_tokens: int = 256):
    prompt = f'### Instruction:\n{instruction}\n\n### Response:\n'
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, top_p=0.9, temperature=0.8)
    gen = out[0][inputs['input_ids'].shape[1]:]
    return tokenizer.decode(gen, skip_special_tokens=True).strip()

print(gpt2_infer(dpo_trainer.model, tokenizer, 'How can I stay disciplined every day?')[:400])